# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['NEPTUNE_API_TOKEN']="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5NDc0ZmNhNi0wODFlLTRhYTktYjgwZS01MWJkMDMxNWJhNTAifQ=="
os.environ['NEPTUNE_PROJECT']="guilhermemg/icao-nets-training-2"
os.environ['NEPTUNE_NOTEBOOK_ID']="98a391a1-c710-40bd-aaf4-42c31862cbbe"
os.environ['NEPTUNE_NOTEBOOK_PATH']="training/exec_nas_experiment.ipynb"

In [3]:
import os
# disable tensorflow log level infos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # show only errors

import sys
import pandas as pd

if '..' not in sys.path:
    sys.path.insert(0, '..')

In [4]:
from src.m_utils import constants as cts
from src.base.data_loaders.data_loader import DLName
from src.base.gt_loaders.gt_names import GTName
from src.exp_runner import ExperimentRunner

from src.base.experiment.dataset.dataset import Dataset
from src.base.experiment.evaluation.model_evaluator import DataSource, DataPredSelection
from src.base.experiment.training.base_models import BaseModel
from src.base.experiment.training.optimizers import Optimizer

from src.m_utils.stl_approach import STLApproach
from src.m_utils.mtl_approach import MTLApproach
from src.m_utils.nas_mtl_approach import NAS_MTLApproach

 ==> Restrict GPU memory growth: True


# Start Network runner

In [5]:
DATASET = Dataset.MNIST
APPROACH = NAS_MTLApproach.APPROACH_2

kwargs = { 
    'use_neptune': False,
    'exp_params' : {
        'name': 'neural_arch_search',
        'description': 'NAS with Approach 2 Fixed',
        'tags': [f'{DATASET.value["name"]}'],
        'src_files': ["../src/**/*.py"]
    },
    'properties': {
        'approach': APPROACH,
        'dataset': DATASET,
        'tasks': DATASET.value['tasks'],
        'balance_input_data': False,
        'train_model': True,
        'save_trained_model': True,
        'exec_nas': True,
        'orig_model_experiment_id': '',
        'sample_training_data': False,
        'sample_prop': 1.0
    },
    'nas_params': {
        #'max_blocks_per_branch': 5,
        #'n_child_epochs': N_CHILD_EPOCHS,
        #'controller_epochs': CONTROLLER_EPOCHS,
        #'controller_batch_size': 64,
        #'n_trials': N_TRIALS,

        'controller_sampling_epochs': 3,
        'samples_per_controller_epochs': 3,
        'controller_training_epochs': 2,
        'architecture_training_epochs': 2,
        'controller_loss_alpha': 0.9
    },
    'controller_params': {
        'controller_lstm_dim': 100,
        'controller_optimizer': Optimizer.ADAM,
        'controller_learning_rate': 0.01,
        'controller_decay': 0.1,
        'controller_momentum': 0.0,
        'controller_use_predictor': True
    },
    'mlp_params': {
        'max_architecture_length': 5,
        'min_task_group_size': 3,
        'mlp_base_model': BaseModel.MOBILENET_V2,
        'mlp_n_epochs': 50,
        'mlp_batch_size': 64,
        'mlp_early_stopping': 50,
        'mlp_optimizer': Optimizer.ADAMAX,
        'mlp_learning_rate': 1e-3,
        'mlp_decay': 0.0,
        'mlp_momentum': 0.0,
        'mlp_dropout': 0.3,
        'mlp_loss_function': 'sparse_categorical_crossentropy',
        'mlp_one_shot': True
    }
}


runner = ExperimentRunner(**kwargs)

-------------------- Init ExperimentRunner -------------------
---------------------------
Parent Process ID: 76168
Process ID: 77526
---------------------------
-----
Use Neptune:  False
-----
-------------------
Args: 
{'controller_params': {'controller_decay': 0.1,
                       'controller_learning_rate': 0.01,
                       'controller_lstm_dim': 100,
                       'controller_momentum': 0.0,
                       'controller_optimizer': <Optimizer.ADAM: 'Adam'>,
                       'controller_use_predictor': True},
 'exp_params': {'description': 'NAS with Approach 2 Fixed',
                'name': 'neural_arch_search',
                'src_files': ['../src/**/*.py'],
                'tags': ['mnist']},
 'mlp_params': {'max_architecture_length': 5,
                'min_task_group_size': 3,
                'mlp_base_model': <BaseModel.MOBILENET_V2: {'name': 'mobilnet_v2', 'target_size': (224, 224), 'prep_function': <function preprocess_input at 0x7f8

# Load Data

In [6]:
runner.load_training_data()

-------------------- load training data -------------------
Loading data
TrainData.shape: (48000, 11)
ValidationData.shape: (12000, 11)
TestData.shape: (10000, 11)
Data loaded


In [7]:
runner.train_data.head()

,img_name,n_0,n_1,n_2,n_3,n_4,n_5,n_6,n_7,n_8,n_9
0,/home/guilherme/data1/Dropbox/Link to Desktop/...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,/home/guilherme/data1/Dropbox/Link to Desktop/...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,/home/guilherme/data1/Dropbox/Link to Desktop/...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,/home/guilherme/data1/Dropbox/Link to Desktop/...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,/home/guilherme/data1/Dropbox/Link to Desktop/...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# <font color='red'>Producing Fake Data</font>

In [8]:
runner.produce_fake_data()

-------------------- producing fake data for experimental purposes -------------------
fake_train_data.shape: (500, 11)
fake_validation_data_df.shape: (100, 11)
fake_test_data_df.shape: (50, 11)


# Data Generators

In [9]:
runner.setup_data_generators()

-------------------- setup data generators -------------------
Starting data generators
Found 500 validated image filenames.
Found 100 validated image filenames.
Found 50 validated image filenames.
TOTAL: 650

Logging class indices
 .. MTL model not logging class indices!

Using benchmarking dataset. Not logging class labels!


# Setup Experiment

In [10]:
runner.setup_experiment()

-------------------- create experiment -------------------
Not using Neptune


# Labels Distribution

In [11]:
%%capture
runner.summary_labels_dist()

# Neural Architecture Search - v2

In [12]:
best_archs_list = runner.run_neural_architecture_search_v2()

-------------------- run neural architecture search -------------------
Vocab: {1: ('n_denses_0', 1), 2: ('n_denses_1', 1), 3: ('n_denses_2', 1), 4: ('n_denses_3', 1), 5: ('n_denses_0', 2), 6: ('n_denses_1', 2), 7: ('n_denses_2', 2), 8: ('n_denses_3', 2), 9: ('n_denses_0', 3), 10: ('n_denses_1', 3), 11: ('n_denses_2', 3), 12: ('n_denses_3', 3), 13: ('n_denses_0', 4), 14: ('n_denses_1', 4), 15: ('n_denses_2', 4), 16: ('n_denses_3', 4), 17: ('n_denses_0', 5), 18: ('n_denses_1', 5), 19: ('n_denses_2', 5), 20: ('n_denses_3', 5)}
Controller classes:  21
Model path:  trained_model
----
Checking model existence locally...
Training a new model! Not checking model existence
----
------------------------------
Checking GPU availability
 ..GPU is available!
------------------------------
------------------------------------------------------------------
                       CONTROLLER EPOCH: 0
------------------------------------------------------------------
GENERATING ARCHITECTURE SAMPLES...


/home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '



Epoch 00002: val_loss did not improve from 0.72360
..Loading best model
..Checkpoint weights loaded
Testing Trained Model
Predicting labels....
2/2 [==============================] - 1s 321ms/step
Prediction finished!
  Task  0: n_0             | EER_mean: 0.47920 | EER_interp: 0.47040 | ACC: 0.53000
  Task  1: n_1             | EER_mean: 0.54720 | EER_interp: 0.53960 | ACC: 0.46000
  Task  2: n_2             | EER_mean: 0.53700 | EER_interp: 0.51850 | ACC: 0.48000
  Task  3: n_3             | EER_mean: 0.51020 | EER_interp: 0.49040 | ACC: 0.51000
  Task  4: n_4             | EER_mean: 0.44680 | EER_interp: 0.44040 | ACC: 0.56000
  Task  5: n_5             | EER_mean: 0.57780 | EER_interp: 0.57070 | ACC: 0.43000
  Task  6: n_6             | EER_mean: 0.63040 | EER_interp: 0.63000 | ACC: 0.37000
  Task  7: n_7             | EER_mean: 0.52270 | EER_interp: 0.52920 | ACC: 0.47000
  Task  8: n_8             | EER_mean: 0.45450 | EER_interp: 0.44940 | ACC: 0.55000
  Task  9: n_9           

KeyboardInterrupt: 

In [15]:
n_obj = runner.nas_object
n_obj.controller_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 1, 4)]       0                                            
__________________________________________________________________________________________________
rnn (RNN)                       (None, 1, 100)       42000       main_input[0][0]                 
__________________________________________________________________________________________________
main_output (Dense)             (None, 1, 21)        2121        rnn[0][0]                        
__________________________________________________________________________________________________
predictor_output (Dense)        (None, 1, 1)         101         rnn[0][0]                        
Total params: 44,222
Trainable params: 44,222
Non-trainable params: 0
________________________

In [30]:
import numpy as np
model = n_obj.controller_model
pred = model.predict(np.zeros((1,1,4)))
print(pred[0])

[[[0.05078977 0.04950244 0.04563015 0.04650212 0.05040184 0.04982164
   0.04426701 0.04820668 0.04481673 0.04549101 0.04984645 0.04723112
   0.04653168 0.04744488 0.04400509 0.04783521 0.04671215 0.04611595
   0.05151049 0.04490633 0.0524312 ]]]


In [37]:
x_data = np.zeros((1,1,4))
y_data = np.zeros((1,1,21))

H = model.fit({'main_input': x_data},
                  {'main_output': y_data},
                  epochs=3,
                  batch_size=1,
                  verbose=1)

print(pd.DataFrame(data=H.history).head())

Epoch 1/3
1/1 [==============================] - 0s 6ms/step - loss: 4.8441e-07 - main_output_loss: 4.8441e-07 - predictor_output_loss: 0.0000e+00
Epoch 2/3
1/1 [==============================] - 0s 4ms/step - loss: 3.6330e-07 - main_output_loss: 3.6330e-07 - predictor_output_loss: 0.0000e+00
Epoch 3/3
1/1 [==============================] - 0s 5ms/step - loss: 4.8441e-07 - main_output_loss: 4.8441e-07 - predictor_output_loss: 0.0000e+00
           loss  main_output_loss  predictor_output_loss
0  4.844060e-07      4.844060e-07                    0.0
1  3.633045e-07      3.633045e-07                    0.0
2  4.844060e-07      4.844060e-07                    0.0


# Create Model with Best Architecture Found

In [ ]:
#best_arch = {'n_denses_0':2,'n_denses_1':2,'n_denses_2':2,'n_denses_3':2}
best_arch = best_archs_list[0]['Decoded architecture'] if best_archs_list is not None else None

In [ ]:
runner.create_model(best_arch)

# Visualize Model

In [ ]:
%%capture
runner.visualize_model(outfile_path=f"training/figs/nas/nas_model_{APPROACH.name}.png")

In [ ]:
%%capture
runner.model_summary()

# Training Model

In [ ]:
runner.train_model()

# Plots

In [ ]:
runner.draw_training_history()

# Load Best Model

In [ ]:
runner.load_best_model()

# Saving Trained Model

In [ ]:
runner.save_model()

# Test Trained Model

## Validation Split

In [ ]:
runner.set_model_evaluator_data_src(DataSource.VALIDATION)
runner.test_model(verbose=False)

## Test Split

In [ ]:
runner.set_model_evaluator_data_src(DataSource.TEST)
runner.test_model(verbose=False)

# Visualize Model Classification

# Finishing Experiment Manager

In [ ]:
runner.finish_experiment()